# <b>Locating a Healthy Restaurant in Chicago</b>

### Introduction/Business Problem 

Chicago is widely known for its music, its theaters, the lake Michigan and specially for its restaurants, with Deep Dish Pizza being the most recognized of all its dishes. However, today people are searching for more healthy options and clearly US and Chicago don't offer a lot of them, and the ones that are already installed are very expensive for most of the people, meaning most of them couldn't buy them every day. <br>  <br> This project means to search for zones/neighborhoods in the city of Chicago were a new healthy restaurant must me placed. For that we are going to use data from Foursquare to search the areas with more restaurants and also the kind, or categories, of food that you can find there.

### Data

First of all I will examine the list of neighborhood in Chicago and create a data frame with it

### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Import Chicago Data by Zip Code

In [16]:
#Read data from of Chicago Zip Codes into a Pandas DataFrame
chicago_data=pd.read_html('http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?zip=606')
chicago_data=pd.concat([chicago_data[2], chicago_data[3]]).reset_index()

In [25]:
chicago_data.drop('index',axis=1, inplace=True)
chicago_data

,Zip Code,City,County
0,60601,Chicago,Cook County
1,60602,Chicago,Cook County
2,60603,Chicago,Cook County
3,60604,Chicago,Cook County
4,60605,Chicago,Cook County
5,60606,Chicago,Cook County
6,60607,Chicago,Cook County
7,60608,Chicago,Cook County
8,60609,Chicago,Cook County
9,60610,Chicago,Cook County


In [81]:
!pip install pgeocode
import pgeocode
print("Pgeocode imported.")

Pgeocode imported.


In [47]:
postal_codes=chicago_data['Zip Code'].apply(str)
nomi = pgeocode.Nominatim('US')
lat_lon=nomi.query_postal_code(list(postal_codes))

### 3. Get Latitude an Longitude Values for each Zip Code zone

In [58]:
lat_long=lat_lon[['postal_code','latitude','longitude']].apply(pd.to_numeric)
n_chicago_data=chicago_data.merge(lat_long, left_on='Zip Code', right_on='postal_code').drop('postal_code', axis=1)
n_chicago_data

,Zip Code,City,County,latitude,longitude
0,60601,Chicago,Cook County,41.8858,-87.6181
1,60602,Chicago,Cook County,41.8829,-87.6321
2,60603,Chicago,Cook County,41.8798,-87.6285
3,60604,Chicago,Cook County,41.8785,-87.6330
4,60605,Chicago,Cook County,41.8713,-87.6277
5,60606,Chicago,Cook County,41.8868,-87.6386
6,60607,Chicago,Cook County,41.8721,-87.6578
7,60608,Chicago,Cook County,41.8515,-87.6694
8,60609,Chicago,Cook County,41.8097,-87.6533
9,60610,Chicago,Cook County,41.9033,-87.6336


### 4. Map Each Zip Code zone in Chicago

In [61]:
# create map of Chicago using latitude and longitude values
map_chicago = folium.Map(location=[41.874662, -87.630146], zoom_start=11)

# add markers to map
for lat, lng, label in zip(n_chicago_data['latitude'], n_chicago_data['longitude'], n_chicago_data['Zip Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

### 5. Use the Foursquare API to explore the neighborhoods

In [74]:
CLIENT_ID = 'J0JFQXISBYT1MTD01434KAX0L3ZJBGI0LQQQY3NMFSF2BOH3' # your Foursquare ID
CLIENT_SECRET = 'NHQTBYLPW53GOD5EVLG3PT434BMCOXH4BJI4QKS4D5IOM0JW' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100 
radius = 500
search_query='Restaurant'
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
chicago_venues = getNearbyVenues(names=n_chicago_data['Zip Code'],
                                   latitudes=n_chicago_data['latitude'],
                                   longitudes=n_chicago_data['longitude']
                                  )

In [77]:
print(chicago_venues.shape)
chicago_venues.head()

(3001, 7)


,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,60601,41.8858,-87.6181,Lakeshore East Park,41.886081,-87.617769,Park
1,60601,41.8858,-87.6181,Mariano's,41.885326,-87.618923,Grocery Store
2,60601,41.8858,-87.6181,Maggie Daley Ice Skating Ribbon,41.884093,-87.619374,Skating Rink
3,60601,41.8858,-87.6181,"Radisson Blu Aqua Hotel, Chicago",41.886308,-87.619921,Hotel
4,60601,41.8858,-87.6181,Eggy's,41.885517,-87.618902,Diner


### Filter Venues Category with just Restaurants

In [80]:
chicago_restaurants=chicago_venues[chicago_venues['Venue Category'].str.contains('Restaurant')]
chicago_restaurants.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,60601,41.8858,-87.6181,The Palm Chicago,41.887033,-87.619468,Seafood Restaurant
11,60601,41.8858,-87.6181,Brown Bag Seafood Co.,41.884598,-87.618952,Seafood Restaurant
13,60601,41.8858,-87.6181,Minghin Cuisine,41.885381,-87.619045,Dim Sum Restaurant
16,60601,41.8858,-87.6181,Mezcalina,41.885647,-87.618991,Mexican Restaurant
24,60601,41.8858,-87.6181,Radisson Blu - Filini Restaurant and Bar,41.886385,-87.620102,Italian Restaurant
